In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import scipy.linalg as spl
import matplotlib.pyplot as plt

from learning import lstd, valuefunc
from env import linearenv, controller
from exputils import relerr
from trajgen import quadratic, nonlinear

## TrajDataset class

In [21]:
p = 5
q = 3
T = 10

In [25]:
lqrenv = env.linearenv.random_vanilla_env(p, q, QRratio=10, Anorm=1.0)
xtraj, _, utraj, rtraj, xtraj_, _ = lqrenv.sampletraj(lqrenv.randctrl(), 10, 500)
dataset = trajdata.TrajDataset(xtraj, utraj, rtraj, xtraj_)

In [26]:
icnnvalue = valuefunc.ICNNValueFunc(p, [50, 50, 1])
icnnvalue.learn(dataset, gamma, num_epoch=400, batch_size=64, verbose=True, print_interval=50)

Epoch: 1 	 Training loss: 1962483.8118374601
Epoch: 51 	 Training loss: 482739.31238068745
Epoch: 101 	 Training loss: 256562.70571207273
Epoch: 151 	 Training loss: 57662.38951897027
Epoch: 201 	 Training loss: 191309.2164756614


KeyboardInterrupt: 

In [192]:
p = 5
q = 3
Q, R = np.eye(p), np.eye(q)
sigma_w = 0
sigma_v = 0
gamma = 0.99
A, B = test_utils.random_env(p, q, Anorm=1.05)

In [255]:
from env import linearenv

In [278]:
C = np.eye(p)
Hw = np.eye(p)
Hv = np.eye(p)
lenv = linearenv.LQREnv(A, B, C, Hw, Hv, Q, R, gamma, sigma_w, sigma_v)

In [279]:
ctrl = test_utils.zero_controller(1, q)
num_trajs = 2
x0s = np.random.randn(num_trajs, p)

In [280]:
T = 10
x, y, u, r, x_, y_ = lenv.sampletraj(ctrl, T, x0s=x0s)

In [281]:
ctrl = lambda x: np.zeros(q)
xtraj, utraj, rtraj, xtraj_ = test_utils.sample_multiple_traj(
    A, B, Q, R, ctrl, T, x0=x0s[0], sigma=0, num_traj=num_trajs)

In [282]:
lenvt = lenv.totracking(T)

In [283]:
lenvt.A

array([[-0.1170837 , -0.60677764,  0.24191989, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.18784614,  0.83228807, -0.7567469 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20109958,  0.79525988,  0.06646807, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [284]:
lenv.solve(10)

(array([[ 1.6161897 , -0.12993769,  0.11953259, -0.30421853, -0.59029854],
        [-0.12993769,  1.72399628, -0.25106677, -0.22582142,  0.15138966],
        [ 0.11953259, -0.25106677,  1.24740909,  0.14653036, -0.11536484],
        [-0.30421853, -0.22582142,  0.14653036,  1.57506712,  0.46989366],
        [-0.59029854,  0.15138966, -0.11536484,  0.46989366,  1.7875112 ]]),
 [array([[ 0.17609623,  0.25338951, -0.17650484,  0.06620612,  0.21953959],
         [-0.15693683,  0.1964491 , -0.32825734, -0.20514409, -0.03385657],
         [-0.05481676, -0.57914699,  0.15459652,  0.47626668,  0.13316139]]),
  array([[ 0.17609623,  0.25338951, -0.17650484,  0.06620612,  0.21953959],
         [-0.15693683,  0.1964491 , -0.32825734, -0.20514409, -0.03385657],
         [-0.05481676, -0.57914699,  0.15459652,  0.47626668,  0.13316139]]),
  array([[ 0.17609623,  0.25338951, -0.17650484,  0.06620612,  0.21953959],
         [-0.15693683,  0.1964491 , -0.32825734, -0.20514409, -0.03385657],
         [-